本节介绍如何使用飞桨在多GPU上训练神经网络模型，在启动训练前，加载数据和网络结构的代码部分均不变。

In [1]:
# 加载相关库
import os
import random
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Conv2D, Pool2D, FC
import numpy as np
from PIL import Image

import gzip
import json

# 定义数据集读取器
def load_data(mode='train'):

    # 读取数据文件
    datafile = './work/mnist.json.gz'
    print('loading mnist dataset from {} ......'.format(datafile))
    data = json.load(gzip.open(datafile))
    # 读取数据集中的训练集，验证集和测试集
    train_set, val_set, eval_set = data

    # 数据集相关参数，图片高度IMG_ROWS, 图片宽度IMG_COLS
    IMG_ROWS = 28
    IMG_COLS = 28
    # 根据输入mode参数决定使用训练集，验证集还是测试
    if mode == 'train':
        imgs = train_set[0]
        labels = train_set[1]
    elif mode == 'valid':
        imgs = val_set[0]
        labels = val_set[1]
    elif mode == 'eval':
        imgs = eval_set[0]
        labels = eval_set[1]
    # 获得所有图像的数量
    imgs_length = len(imgs)
    # 验证图像数量和标签数量是否一致
    assert len(imgs) == len(labels), \
          "length of train_imgs({}) should be the same as train_labels({})".format(
                  len(imgs), len(labels))

    index_list = list(range(imgs_length))

    # 读入数据时用到的batchsize
    BATCHSIZE = 100

    # 定义数据生成器
    def data_generator():
        # 训练模式下，打乱训练数据
        if mode == 'train':
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        # 按照索引读取数据
        for i in index_list:
            # 读取图像和标签，转换其尺寸和类型
            img = np.reshape(imgs[i], [1, IMG_ROWS, IMG_COLS]).astype('float32')
            label = np.reshape(labels[i], [1]).astype('int64')
            imgs_list.append(img) 
            labels_list.append(label)
            # 如果当前数据缓存达到了batch size，就返回一个批次数据
            if len(imgs_list) == BATCHSIZE:
                yield np.array(imgs_list), np.array(labels_list)
                # 清空数据缓存列表
                imgs_list = []
                labels_list = []

        # 如果剩余数据的数目小于BATCHSIZE，
        # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
        if len(imgs_list) > 0:
            yield np.array(imgs_list), np.array(labels_list)

    return data_generator


# 定义模型结构
class MNIST(fluid.dygraph.Layer):
     def __init__(self, name_scope):
         super(MNIST, self).__init__(name_scope)
         name_scope = self.full_name()
         # 定义卷积层，输出通道20，卷积核大小为5，步长为1，padding为2，使用relu激活函数
         self.conv1 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义池化层，池化核为2，采用最大池化方式
         self.pool1 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         # 定义卷积层，输出通道20，卷积核大小为5，步长为1，padding为2，使用relu激活函数
         self.conv2 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义池化层，池化核为2，采用最大池化方式
         self.pool2 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         # 定义全连接层，输出节点数为10，激活函数使用softmax
         self.fc = FC(name_scope, size=10, act='softmax')
         
    # 定义网络的前向计算过程
     def forward(self, inputs):
         x = self.conv1(inputs)
         x = self.pool1(x)
         x = self.conv2(x)
         x = self.pool2(x)
         x = self.fc(x)
         return x

# 单GPU训练

从前几节的训练来看，我们无论是训练房价预测模型还是MNIST手写字符识别模型，训练好一个模型不会超过十分钟，主要原因是我们所使用的神经网络比较简单。但现实生活中，我们可能会遇到更复杂的机器学习、深度学习任务，需要运算速度更高的硬件（GPU、TPU），甚至同时使用多个机器共同训练一个任务（多卡训练和多机训练）。

飞桨动态图通过fluid.dygraph.guard(place=None)里的place参数，设置在GPU上训练还是CPU上训练，比如：
```
with fluid.dygraph.guard(place=fluid.CPUPlace())　#设置使用CPU资源训神经网络。
with fluid.dygraph.guard(place=fluid.CUDAPlace(0))　#设置使用GPU资源训神经网络，默认使用机器的第一个GPU。
```

In [2]:
#仅前3行代码有所变化，在使用GPU时，可以将use_gpu变量设置成True
use_gpu = False
place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()

with fluid.dygraph.guard(place):
    model = MNIST("mnist")
    model.train()
    #调用加载数据的函数
    train_loader = load_data('train')
    
    #四种优化算法的设置方案，可以逐一尝试效果
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.01)
    
    EPOCH_NUM = 2
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程
            predict = model(image)
            
            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

loading mnist dataset from ./work/mnist.json.gz ......
epoch: 0, batch: 0, loss is: [2.8070326]
epoch: 0, batch: 200, loss is: [0.5153172]
epoch: 0, batch: 400, loss is: [0.44996807]
epoch: 1, batch: 0, loss is: [0.18519798]
epoch: 1, batch: 200, loss is: [0.30971336]
epoch: 1, batch: 400, loss is: [0.22442476]


# 分布式训练

在工业实践中，许多较复杂的任务需要使用更强大的模型。强大模型加上海量的训练数据，经常导致模型训练耗时严重。比如在计算机视觉分类任务中，训练一个在ImageNet数据集上精度表现良好的模型，大概需要一周的时间，因为我们需要不断尝试各种优化的思路和方案。如果每次训练均要耗时1周，这会大大降低模型迭代的速度。在机器资源充沛的情况下，我们可以采用分布式训练，大部分模型的训练时间可压缩到小时级别。

分布式训练有两种实现模式：模型并行和数据并行。


## 1. 模型并行

模型并行是将一个网络模型拆分为多份，拆分后的模型分到多个设备上（GPU）训练，每个设备的训练数据是相同的。
模型并行的方式一般适用于：
1. 模型架构过大，完整的模型无法放入单个GPU。2012年ImageNet大赛的冠军模型AlexNet是模型并行的典型案例。由于当时GPU内存较小，单个GPU不足以承担AlexNet。研究者将AlexNet拆分为两部分放到两个GPU上并行训练。

2. 网络模型的设计结构可以并行化时，采用模型并行的方式。例如在计算机视觉目标检测任务中，一些模型（YOLO9000）的边界框回归和类别预测是独立的，可以将独立的部分分在不同的设备节点上完成分布式训练。

说明：当前GPU硬件技术快速发展，深度学习使用的主流GPU的内存已经足以满足大多数的网络模型需求，所以大多数情况下使用数据并行的方式。


## 2. 数据并行

数据并行与模型并行不同，数据并行每次读取多份数据，读取到的数据输入给多个设备（GPU）上的模型，每个设备上的模型是完全相同的。数据并行的方式与众人拾柴火焰高的道理类似，如果把训练数据比喻为砖头，把一个设备（GPU）比喻为一个人，那单GPU训练就是一个人在搬砖，多GPU训练就是多个人同时搬砖，每次搬砖的数量倍数增加，效率呈倍数提升。但是注意到，每个设备的模型是完全相同的，但是输入数据不同，每个设备的模型计算出的梯度是不同的，如果每个设备的梯度更新当前设备的模型就会导致下次训练时，每个模型的参数都不同了，所以我们还需要一个梯度同步机制，保证每个设备的梯度是完全相同的。

数据并行中有一个参数管理服务器（parameter server）收集来自每个设备的梯度更新信息，并计算出一个全局的梯度更新。当参数管理服务器收到来自训练设备的梯度更新请求时，统一更新模型的梯度。

飞桨有便利的数据并行训练方式，仅改动几行代码即可实现多GPU训练，如果想了解飞桨数据并行的基本思想，可以参考官网文档-[https://www.paddlepaddle.org.cn/documentation/docs/zh/user_guides/howto/training/cluster_howto.html](https://www.paddlepaddle.org.cn/documentation/docs/zh/user_guides/howto/training/cluster_howto.html)。




用户只需要对程序进行简单修改，即可实现在多GPU上并行训练。飞桨采用数据并行的实现方式，在训练前，需要配置如下参数：

* 1.从环境变量获取设备的ID，并指定给CUDAPlace
```
  device_id = fluid.dygraph.parallel.Env().dev_id
  place = fluid.CUDAPlace(device_id)
```
* 2.对定义的网络做预处理，设置为并行模式 
```
  strategy = fluid.dygraph.parallel.prepare_context() ## 新增
  model = MNIST("mnist")
  model = fluid.dygraph.parallel.DataParallel(model, strategy)  ## 新增
 ```
* 3.定义多GPU训练的reader，将每批次的数据平分到每个GPU上
```
  valid_loader = paddle.batch(paddle.dataset.mnist.test(), batch_size=16, drop_last=true)
  valid_loader = fluid.contrib.reader.distributed_batch_reader(valid_loader)
```
* 4.收集每批次训练数据的loss，并聚合参数的梯度
```
  avg_loss = mnist.scale_loss(avg_loss)  ## 新增
  avg_loss.backward()
  mnist.apply_collective_grads()         ## 新增
```

完整程序如下所示。

In [3]:
def train_multi_gpu():
    
    ##修改1-从环境变量获取使用GPU的序号
    place = fluid.CUDAPlace(fluid.dygraph.parallel.Env().dev_id)

    with fluid.dygraph.guard(place):
    
        ##修改2-对原模型做并行化预处理
        strategy = fluid.dygraph.parallel.prepare_context()
        model = MNIST("mnist")
        model = fluid.dygraph.parallel.DataParallel(model, strategy)

        model.train()

        #调用加载数据的函数
        train_loader = load_data('train')
        ##修改3-多GPU数据读取，必须确保每个进程读取的数据是不同的
        train_loader = fluid.contrib.reader.distributed_batch_reader(train_loader)

        optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
        EPOCH_NUM = 5
        for epoch_id in range(EPOCH_NUM):
            for batch_id, data in enumerate(train_loader()):
                #准备数据
                image_data, label_data = data
                image = fluid.dygraph.to_variable(image_data)
                label = fluid.dygraph.to_variable(label_data)

                predict = model(image)

                loss = fluid.layers.square_error_cost(predict, label)
                avg_loss = fluid.layers.mean(loss)

                # 修改4-多GPU训练需要对Loss做出调整，并聚合不同设备上的参数梯度
                avg_loss = mnist.scale_loss(avg_loss)
                avg_loss.backward()
                model.apply_collective_grads()
                # 最小化损失函数，清除本次训练的梯度
                optimizer.minimize(avg_loss)
                model.clear_gradients()
                
                if batch_id % 200 == 0:
                    print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

启动多GPU的训练，需要在命令行设置一些参数变量。打开终端，运行如下命令：

```
$ python -m paddle.distributed.launch --selected_gpus=0,1,2,3 --log_dir ./mylog train_multi_gpu.py
```
- paddle.distributed.launch表示启动分布式运行。
- 通过selected_gpus设置使用的GPU的序号。当前机器需要是多GPU卡的机器，通过命令watch nvidia-smi可以查看GPU的序号。
- log_dir用于存放训练的log，如果不设置，每个GPU上的训练信息都会打印到屏幕。
- 多GPU运行的脚本是：train_multi_gpu.py，包含上述修改过的train_multi_gpu()函数。

训练完成后，程序会在指定的./mylog文件夹下产生四个worklog文件。每个文件存放对应设备的训练过程日志，其中worklog.0的内容如下：

In [ ]:
grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
dev_id 0
I1104 06:25:04.377323 31961 nccl_context.cc:88] worker: 127.0.0.1:6171 is not ready, will retry after 3 seconds...
I1104 06:25:07.377645 31961 nccl_context.cc:127] init nccl context nranks: 3 local rank: 0 gpu id: 1↩
W1104 06:25:09.097079 31961 device_context.cc:235] Please NOTE: device: 1, CUDA Capability: 61, Driver API Version: 10.1, Runtime API Version: 9.0
W1104 06:25:09.104460 31961 device_context.cc:243] device: 1, cuDNN Version: 7.5.
start data reader (trainers_num: 3, trainer_id: 0)
epoch: 0, batch_id: 10, loss is: [0.47507238]
epoch: 0, batch_id: 20, loss is: [0.25089613]
epoch: 0, batch_id: 30, loss is: [0.13120805]
epoch: 0, batch_id: 40, loss is: [0.12122715]
epoch: 0, batch_id: 50, loss is: [0.07328521]
epoch: 0, batch_id: 60, loss is: [0.11860339]
epoch: 0, batch_id: 70, loss is: [0.08205047]
epoch: 0, batch_id: 80, loss is: [0.08192863]
epoch: 0, batch_id: 90, loss is: [0.0736289]
epoch: 0, batch_id: 100, loss is: [0.08607423]
start data reader (trainers_num: 3, trainer_id: 0)
epoch: 1, batch_id: 10, loss is: [0.07032011]
epoch: 1, batch_id: 20, loss is: [0.09687119]
epoch: 1, batch_id: 30, loss is: [0.0307216]
epoch: 1, batch_id: 40, loss is: [0.03884467]
epoch: 1, batch_id: 50, loss is: [0.02801813]
epoch: 1, batch_id: 60, loss is: [0.05751991]
epoch: 1, batch_id: 70, loss is: [0.03721186]
.....